# Optimize the Model

In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Trial 1: Adjust Input Data - Drop More Columns

In [18]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

# Review
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Replace rare occurrences in APPLICATION_TYPE and CLASSIFICATION with 'Other'
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(app_type_counts[app_type_counts < 1000].index)
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

class_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(class_counts[class_counts < 1500].index)
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

In [20]:
# Convert categorical data to numeric
application_df = pd.get_dummies(application_df, dtype=float)


In [ ]:
# Split our preprocessed data into features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"], axis=1).values

In [21]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [22]:
# Create a StandardScaler instance and scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Define and compile the neural network model
nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=len(X_train[0])))

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [24]:
# Compile the model
nn_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
fit_model_1 = nn_1.fit(X_train_scaled, y_train, epochs=100, batch_size=64, validation_split=0.2)


Epoch 1/100
322/322 [==============================] - 0s 751us/step - loss: 0.5757 - accuracy: 0.7205 - val_loss: 0.5749 - val_accuracy: 0.7207
Epoch 2/100
322/322 [==============================] - 0s 475us/step - loss: 0.5581 - accuracy: 0.7319 - val_loss: 0.5689 - val_accuracy: 0.7221
Epoch 3/100
322/322 [==============================] - 0s 457us/step - loss: 0.5546 - accuracy: 0.7320 - val_loss: 0.5647 - val_accuracy: 0.7234
Epoch 4/100
322/322 [==============================] - 0s 461us/step - loss: 0.5514 - accuracy: 0.7337 - val_loss: 0.5672 - val_accuracy: 0.7226
Epoch 5/100
322/322 [==============================] - 0s 460us/step - loss: 0.5505 - accuracy: 0.7337 - val_loss: 0.5637 - val_accuracy: 0.7240
Epoch 6/100
322/322 [==============================] - 0s 484us/step - loss: 0.5492 - accuracy: 0.7354 - val_loss: 0.5705 - val_accuracy: 0.7256
Epoch 7/100
322/322 [==============================] - 0s 506us/step - loss: 0.5487 - accuracy: 0.7351 - val_loss: 0.5669 - val_ac

In [25]:
# Evaluate the model
model_loss_1, model_accuracy_1 = nn_1.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Trial 1 - Loss: {model_loss_1}, Accuracy: {model_accuracy_1}")

268/268 - 0s - loss: 0.5606 - accuracy: 0.7257 - 84ms/epoch - 315us/step
Trial 1 - Loss: 0.5605568289756775, Accuracy: 0.7257142663002014


In [33]:
# Save the model
nn_1.save('./Outputs/AlphabetSoupCharity_Trial1.h5')

## Trial 2: Add More Hidden Layers

In [26]:
# Define and compile the neural network model with more hidden layers
nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=len(X_train[0])))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Fourth hidden layer
nn_2.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [27]:
# Compile the model
nn_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
fit_model_2 = nn_2.fit(X_train_scaled, y_train, epochs=100, batch_size=64, validation_split=0.2)


Epoch 1/100
322/322 [==============================] - 0s 818us/step - loss: 0.5744 - accuracy: 0.7227 - val_loss: 0.5753 - val_accuracy: 0.7221
Epoch 2/100
322/322 [==============================] - 0s 577us/step - loss: 0.5563 - accuracy: 0.7324 - val_loss: 0.5671 - val_accuracy: 0.7246
Epoch 3/100
322/322 [==============================] - 0s 638us/step - loss: 0.5523 - accuracy: 0.7333 - val_loss: 0.5732 - val_accuracy: 0.7230
Epoch 4/100
322/322 [==============================] - 0s 561us/step - loss: 0.5513 - accuracy: 0.7341 - val_loss: 0.5656 - val_accuracy: 0.7230
Epoch 5/100
322/322 [==============================] - 0s 565us/step - loss: 0.5494 - accuracy: 0.7357 - val_loss: 0.5686 - val_accuracy: 0.7223
Epoch 6/100
322/322 [==============================] - 0s 550us/step - loss: 0.5485 - accuracy: 0.7355 - val_loss: 0.5658 - val_accuracy: 0.7246
Epoch 7/100
322/322 [==============================] - 0s 578us/step - loss: 0.5476 - accuracy: 0.7367 - val_loss: 0.5675 - val_ac

In [28]:
# Evaluate the model
model_loss_2, model_accuracy_2 = nn_2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Trial 2 - Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

268/268 - 0s - loss: 0.5676 - accuracy: 0.7251 - 73ms/epoch - 273us/step
Trial 2 - Loss: 0.5676242709159851, Accuracy: 0.7251312136650085


In [34]:
# Save the model
nn_2.save('./Outputs/AlphabetSoupCharity_Trial2.h5')

## Trial 3: Different Activation Functions

In [29]:
# Define and compile the neural network model with different activation functions
nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=128, activation='tanh', input_dim=len(X_train[0])))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=64, activation='tanh'))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=32, activation='tanh'))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [30]:
# Compile the model
nn_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
fit_model_3 = nn_3.fit(X_train_scaled, y_train, epochs=100, batch_size=64, validation_split=0.2)

Epoch 1/100
322/322 [==============================] - 0s 759us/step - loss: 0.5742 - accuracy: 0.7198 - val_loss: 0.5697 - val_accuracy: 0.7211
Epoch 2/100
322/322 [==============================] - 0s 553us/step - loss: 0.5586 - accuracy: 0.7307 - val_loss: 0.5657 - val_accuracy: 0.7226
Epoch 3/100
322/322 [==============================] - 0s 530us/step - loss: 0.5547 - accuracy: 0.7313 - val_loss: 0.5726 - val_accuracy: 0.7147
Epoch 4/100
322/322 [==============================] - 0s 527us/step - loss: 0.5523 - accuracy: 0.7341 - val_loss: 0.5660 - val_accuracy: 0.7258
Epoch 5/100
322/322 [==============================] - 0s 547us/step - loss: 0.5507 - accuracy: 0.7345 - val_loss: 0.5653 - val_accuracy: 0.7269
Epoch 6/100
322/322 [==============================] - 0s 523us/step - loss: 0.5497 - accuracy: 0.7348 - val_loss: 0.5640 - val_accuracy: 0.7226
Epoch 7/100
322/322 [==============================] - 0s 530us/step - loss: 0.5487 - accuracy: 0.7362 - val_loss: 0.5641 - val_ac

In [31]:
# Evaluate the model
model_loss_3, model_accuracy_3 = nn_3.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Trial 3 - Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

268/268 - 0s - loss: 0.5617 - accuracy: 0.7245 - 67ms/epoch - 250us/step
Trial 3 - Loss: 0.5616837739944458, Accuracy: 0.7245481014251709


In [35]:
# Save the model
nn_3.save('./Outputs/AlphabetSoupCharity_Trial3.h5')

## Comparison of Trials

In [32]:
print(f"Trial 1 - Loss: {model_loss_1}, Accuracy: {model_accuracy_1}")
print(f"Trial 2 - Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")
print(f"Trial 3 - Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

Trial 1 - Loss: 0.5605568289756775, Accuracy: 0.7257142663002014
Trial 2 - Loss: 0.5676242709159851, Accuracy: 0.7251312136650085
Trial 3 - Loss: 0.5616837739944458, Accuracy: 0.7245481014251709
